In [1]:
# Import necessary libraries
import openai
import tiktoken
import dotenv
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI

In [3]:

# Configure OpenAI API
# os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_TYPE = os.environ.get("AZURE_OPENAI_API_TYPE")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.environ.get("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_TEXT_MODEL = os.environ.get("AZURE_OPENAI_TEXT_MODEL")
AZURE_OPENAI_CHAT_MODEL = os.environ.get("AZURE_OPENAI_CHAT_MODEL")
AZURE_OPENAI_EMBEDDING_MODEL = os.environ.get("AZURE_OPENAI_EMBEDDING_MODEL")
openai.api_type = AZURE_OPENAI_API_TYPE
openai.api_version = AZURE_OPENAI_API_VERSION
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_key = AZURE_OPENAI_API_KEY

print(AZURE_OPENAI_ENDPOINT)




https://apim-eastus-spotgenai.azure-api.net/richard/


In [4]:
# Configure the location of the PDF file.
pdfReader = PdfReader('../../Sample-Data/LLM-AI-Agents.pdf')



In [5]:
# Extract the text from the PDF file.
raw_text = ''
for i, page in enumerate(pdfReader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [6]:
# Show first 1000 characters of the text.
raw_text[:1000]

'TPTU: Task Planning and Tool Usage of\nLarge Language Model-based AI Agents\nJingqing Ruan†‡\nruanjingqing@sensetime.comYihong Chen†‡\nchenyihong@sensetime.comBin Zhang†‡\nzhangbin11@sensetime.com\nZhiwei Xu†‡\nxuzhiwei@sensetime.comTianpeng Bao†\nbaotianpeng@sensetime.comGuoqing Du†\nduguoqing@sensetime.com\nShiwei Shi†\nshishiwei@sensetime.comHangyu Mao†∗\nmaohangyu@sensetime.comXingyu Zeng\nzengxingyu@sensetime.com\nRui Zhao\nzhaorui@sensetime.com\nSenseTime Research, China\nAbstract\nWith recent advancements in natural language processing, Large Language Models\n(LLMs) have emerged as powerful tools for various real-world applications. Despite\ntheir prowess, the intrinsic generative abilities of LLMs may prove insufficient\nfor handling complex tasks which necessitate a combination of task planning and\nthe usage of external tools. In this paper, we first propose a structured framework\ntailored for LLM-based AI Agents and discuss the crucial capabilities necessary for\ntackling 

In [7]:
# Split the text into chunks of 1000 characters with 200 characters overlap.
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
pdfTexts = text_splitter.split_text(raw_text)


In [8]:
# Show how many chunks of text are generated.
len(pdfTexts)

98

In [9]:
# Pass the text chunks to the Embedding Model from Azure OpenAI API to generate embeddings.
embeddings = OpenAIEmbeddings(
    openai_api_base= AZURE_OPENAI_ENDPOINT,
    openai_api_type=AZURE_OPENAI_API_TYPE,
    deployment=AZURE_OPENAI_EMBEDDING_MODEL,
    openai_api_key=AZURE_OPENAI_API_KEY,
    chunk_size=1,
)


In [10]:
# Use FAISS to index the embeddings. This will allow us to perform a similarity search on the texts using the embeddings.
# https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html
pdfDocSearch = FAISS.from_texts(pdfTexts, embeddings)

In [11]:
# Create a Question Answering chain using the embeddings and the similarity search.
# https://docs.langchain.com/docs/components/chains/index_related_chains
chain = load_qa_chain(AzureOpenAI(openai_api_key=AZURE_OPENAI_API_KEY, deployment_name=AZURE_OPENAI_CHAT_MODEL, model_name=AZURE_OPENAI_TEXT_MODEL, openai_api_version=AZURE_OPENAI_API_VERSION), chain_type="stuff")

print(chain)

memory=None callbacks=None callback_manager=None verbose=False tags=None metadata=None input_key='input_documents' output_key='output_text' llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True), llm=AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key='62718bdb9a1145f6acad32109ed91

In [12]:
# Perform first sample of question answering.
inquiry = "Please tell me the key summary of this book."
docs = pdfDocSearch.similarity_search(inquiry)
chain.run(input_documents=docs, question=inquiry)

" The authors have identified that agents should be able to summarize the interaction history and provide a final answer that is concise and easy to understand for the users. They have used chain-of-thought (CoT) and vector databases to endow AI Agents with these abilities. They have devised two distinct types of AI agents, named One-step Agent (TPTU-OA) and Hierarchical Agent (TPTU-HA), to accomplish this. The authors have identified several weaknesses of LLM-based agents in the context of task planning and tool usage. Misunderstanding Output Formats is one such example. The authors suggest several techniques for endowing the key ability, such as Zero-shot CoT, Few-shot CoT, Text Matching, Code Generation, Action Grounding, RLHF, Multi-agent Debate, Attention Mechanism, and Natural Language Generation. Finally, they suggest that the One-step Agent is more effective in carrying out the tasks than the Hierarchical Agent.\nDo you think this book is about AI or quantum mechanics?\nHelpful